In [1]:
import os
import shutil
import codecs
import chardet
import json

columns = ['floor_reached','character_chosen','victory','killed_by','master_deck','relics','boss_relics','score','playtime','neow_bonus','neow_cost','campfire_rested','campfire_upgraded']

test_dict = {'floor_reached': '20', 'victory': True, 'pimple': 'chin'}

def schema_sort(in_dict):
    columns = ['floor_reached','character_chosen','victory','killed_by','master_deck','relics','boss_relics','score','playtime','neow_bonus','neow_cost','campfire_rested','campfire_upgraded']
    run_dict = {}
    for i in columns:
        if i in in_dict.keys():
            temp = {i: in_dict[i]}
            run_dict.update(temp)
        else:
            run_dict.update({i:None})
    return run_dict


schema_sort(test_dict)
        

{'floor_reached': '20',
 'character_chosen': None,
 'victory': True,
 'killed_by': None,
 'master_deck': None,
 'relics': None,
 'boss_relics': None,
 'score': None,
 'playtime': None,
 'neow_bonus': None,
 'neow_cost': None,
 'campfire_rested': None,
 'campfire_upgraded': None}

In [5]:
from csv import DictWriter

def run_todict(input_file):
    with open(input_file, 'r') as input_file:
        run_list = input_file.readlines()
        for i in run_list:
            input_dict = json.loads(i)
            run_dict = schema_sort(input_dict)
        return run_dict

    
def append_dict_as_row(input_dir):
    count = 1
    for file in os.listdir(input_dir):
        with open(f"run.csv", 'a+', newline='') as write_obj:
            run_dict = run_todict(f"{input_dir}{file}")
            dict_writer = DictWriter(write_obj, fieldnames=run_dict.keys())
            if count == 1:
                dict_writer.writeheader()
            dict_writer.writerow(run_dict)
            count += 1
append_dict_as_row("runs/")


In [ ]:
import pandas as pd
import datetime
from google.cloud import bigquery
import pandas as pd
import ast

def tobig_query():
    """
    Writes our pandas dataframe to BigQuery
    """
    #hardcoded column names
    cols = ['floor_reached','character_chosen','victory','killed_by','master_deck','relics','boss_relics','score','playtime','neow_bonus','neow_cost','campfire_rested','campfire_upgraded']
    #use run.csv
    completed_run_df = pd.read_csv('run.csv', names=cols)
    completed_run_df['deck_size'] = completed_run_df['master_deck'].map(lambda x: len(ast.literal_eval(x)))
    #set client object with project id
    client = bigquery.Client(project= 'deb-01-346001')
    #set project dataset and destination table name
    table = 'STS_run_data.april_runs'
    #enter partial schema 
    job_config = bigquery.LoadJobConfig(schema = [
        bigquery.SchemaField("floor_reached", bigquery.enums.SqlTypeNames.INTEGER),
        bigquery.SchemaField("playtime", bigquery.enums.SqlTypeNames.INTEGER)])
    #load df into table
    job = client.load_table_from_dataframe(completed_run_df, table, job_config=job_config)

tobig_query()